In [2]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 63.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=0cb800f32931a2a5513356df4d9f69eeff0149b3d18627d3c31c3b8a4a344727
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName('RF').getOrCreate()

In [7]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
file_path = '/content/drive//MyDrive/'
data=spark.read.csv(file_path +'dog_food - Copy.csv',inferSchema=True,header=True)

In [9]:
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [17]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["A","B","C","D"],outputCol="Independent Features")

In [18]:
output=featureassembler.transform(data)

In [19]:
from pyspark.ml.classification import RandomForestClassifier

In [26]:
rfc=RandomForestClassifier(labelCol='Spoiled',featuresCol='Independent Features')

In [27]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- Independent Features: vector (nullable = true)



In [28]:
final_data=output.select(['Independent Features','Spoiled'])

In [29]:
final_data.show()

+--------------------+-------+
|Independent Features|Spoiled|
+--------------------+-------+
|  [4.0,2.0,12.0,3.0]|    1.0|
|  [5.0,6.0,12.0,7.0]|    1.0|
|  [6.0,2.0,13.0,6.0]|    1.0|
|  [4.0,2.0,12.0,1.0]|    1.0|
|  [4.0,2.0,12.0,3.0]|    1.0|
| [10.0,3.0,13.0,9.0]|    1.0|
|  [8.0,5.0,14.0,5.0]|    1.0|
|  [5.0,8.0,12.0,8.0]|    1.0|
|  [6.0,5.0,12.0,9.0]|    1.0|
|  [3.0,3.0,12.0,1.0]|    1.0|
|  [9.0,8.0,11.0,3.0]|    1.0|
| [1.0,10.0,12.0,3.0]|    1.0|
| [1.0,5.0,13.0,10.0]|    1.0|
| [2.0,10.0,12.0,6.0]|    1.0|
| [1.0,10.0,11.0,4.0]|    1.0|
|  [5.0,3.0,12.0,2.0]|    1.0|
|  [4.0,9.0,11.0,8.0]|    1.0|
|  [5.0,1.0,11.0,1.0]|    1.0|
| [4.0,9.0,12.0,10.0]|    1.0|
|  [5.0,8.0,10.0,9.0]|    1.0|
+--------------------+-------+
only showing top 20 rows



In [30]:
rfc_model=rfc.fit(final_data)

In [31]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0164, 1: 0.0127, 2: 0.9541, 3: 0.0168})